In [ ]:
!pip install spark-nlp==2.6.0 pyspark==2.4.4

     |████████████████████████████████| 133kB 8.8MB/s 
     |████████████████████████████████| 215.7MB 72kB/s 
     |████████████████████████████████| 204kB 35.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130389 sha256=0100240126007110c6dd8f809bc98c41ae942258581317ef18fb7c26b58e28c8
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [ ]:
import os
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [ ]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [ ]:
!java -version

openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-8u282-b08-0ubuntu1~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.listdir()

['.DS_Store',
 'Untitled0.ipynb',
 'covid_dataset_with_ids',
 'Untitled1.ipynb',
 'dataset_final_with_uid',
 'new_preprocessed_covid_dataset',
 'new_preprocessed_dataset_final',
 'text_preprocessing.ipynb',
 'sar_sent_covid_dataset',
 'sar_sent_dataset_final',
 'USE SE 512',
 'BERT SE 128',
 'samp_train_tsne.csv',
 'samp_test_tsne.csv',
 'small_bert_bert_en_uncased_L-12_H-128_A-2_1']

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
spark = sparknlp.start() 

In [ ]:
df = spark.read.option("header", True).option("multiLine",True).option("inferSchema",True).csv("/content/drive/MyDrive/Sentence Embeddings/sar_sent_covid_dataset")
df1 = spark.read.option("header", True).option("inferSchema",True).parquet("/content/drive/MyDrive/Sentence Embeddings/sar_sent_dataset_final")
# df2 = spark.read.option("header", True).option("multiLine",True).option("inferSchema",True).csv("/content/drive/MyDrive/Sentence Embeddings/all_combined_final")

In [ ]:
print("C1: "+str(df.count())+", C2: "+str(df1.count()))

C1: 52020, C2: 1581420


DataFrame[id: string, text: string, final_sent_class: string, sarcasm: string]

In [ ]:
document = DocumentAssembler().setInputCol("text").setOutputCol("document")
# use = UniversalSentenceEncoder.pretrained().setInputCols(["document"]).setOutputCol("USE_pre_text_all_upd_embed")
sentenceDetector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")
bert = BertSentenceEmbeddings.pretrained("sent_small_bert_L12_128", "en").setInputCols("sentence").setOutputCol("sentence_embeddings")
# use_clf_pipeline = Pipeline(
#     stages = [
#         document,
#         use
#     ])
bert_clf_pipeline = Pipeline(
    stages = [
        document,
        sentenceDetector,
        bert
    ])

sent_small_bert_L12_128 download started this may take some time.
Approximate size to download 23.4 MB
[OK!]


In [ ]:
df_bert = bert_clf_pipeline.fit(df).transform(df)
df1_bert = bert_clf_pipeline.fit(df1).transform(df1)

In [ ]:
df_bert.printSchema()

root
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- final_sent_class: string (nullable = true)
 |-- sarcasm: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- meta

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.linalg import VectorUDT, DenseVector

ud_f = F.udf(lambda r : DenseVector(r),VectorUDT())

In [ ]:
res_df = df_bert.select("id",F.explode("sentence_embeddings").alias("embeddings"),"sarcasm","final_sent_class")
res_df = res_df.select("id","embeddings.embeddings","sarcasm","final_sent_class")

In [ ]:
res_df1 = df1_bert.select("id",F.explode("sentence_embeddings").alias("embeddings"),"sarcasm","final_sent_class")
res_df1 = res_df1.select("id","embeddings.embeddings","sarcasm","final_sent_class")

In [ ]:
res_df = res_df.withColumn("BERT_embed",ud_f(F.col("embeddings")))
res_df1 = res_df1.withColumn("BERT_embed",ud_f(F.col("embeddings")))

In [ ]:
res_df = res_df.drop("embeddings")
res_df1 = res_df1.drop("embeddings")

In [ ]:
res_df = res_df.select("id","BERT_embed","sarcasm","final_sent_class")
res_df1 = res_df1.select("id","BERT_embed","sarcasm","final_sent_class")

In [ ]:
res_df1

DataFrame[id: string, BERT_embed: vector, sarcasm: string, final_sent_class: string]

In [ ]:
x = [0.1971338838338852,-0.26955828070640564,0.23428303003311157,0.30238214135169983,0.3969629406929016,-0.18533313274383545,-0.08664914965629578,0.20012396574020386,-0.9772298336029053,-0.3001679480075836,-0.4532845914363861,0.6719492077827454,-0.09347500652074814,-1.0238310098648071,-0.2507983148097992,-1.0146217346191406,-0.9772282838821411,-0.34891268610954285,0.9876363277435303,0.05500255897641182,-1.0992047786712646,1.0956398248672485,1.3010482788085938,-0.9334847927093506,0.6957373023033142,0.08280075341463089,0.29108941555023193,0.35715925693511963,1.0095542669296265,-0.059544000774621964,0.3823041319847107,1.0512096881866455,-2.3096344470977783,-0.4323384165763855,-1.7843492031097412,0.5367324948310852,-0.47537511587142944,-0.9508094787597656,0.41271868348121643,-0.3713393211364746,-0.19445830583572388,0.35403621196746826,-0.45899713039398193,-0.17753219604492188,-0.3613598346710205,-2.0063817501068115,0.2819390594959259,0.05349244922399521,0.1349763572216034,-0.8843961954116821,1.0906224250793457,0.34021317958831787,2.2164011001586914,-0.27533939480781555,-1.5271682739257812,2.493889093399048,1.4754620790481567,0.06727715581655502,0.08624772727489471,0.3053363561630249,-1.06379234790802,0.16802014410495758,0.5333545207977295,-0.5820779800415039,0.4201391935348511,-0.42006829380989075,-0.3670503795146942,0.3314417004585266,1.0510187149047852,0.728222668170929,-0.28452157974243164,0.07638781517744064,-0.5207154750823975,0.09679098427295685,-0.11360679566860199,0.7749133110046387,0.12946471571922302,-2.203493118286133,-0.9760034084320068,0.11325308680534363,-0.23695765435695648,0.20268100500106812,-0.37719860672950745,-0.268322229385376,-0.5990903377532959,0.512627363204956,-0.0012546314392238855,1.0604732036590576,1.5042392015457153,-0.19645777344703674,-0.5520015358924866,1.7667386531829834,-0.30886876583099365,-1.0844216346740723,-0.3834240138530731,-0.3079751431941986,-0.08624021708965302,0.4988824129104614,0.25400590896606445,0.4737401604652405,0.5451825857162476,0.49143335223197937,-0.6702772378921509,-0.240690678358078,1.4208669662475586,0.5565597414970398,-0.5170107483863831,-0.24662858247756958,-0.0371110625565052,0.007880141027271748,-4.15882682800293,0.006242822390049696,0.4874117076396942,0.7323100566864014,1.1704908609390259,0.19226087629795074,-0.2658025324344635,0.8133059740066528,-0.025646917521953583,-1.4784740209579468,0.27270257472991943,-0.8595114946365356,0.9264666438102722,0.5400975346565247,0.853740930557251,-0.10915511101484299,-0.9748389720916748,-0.06898687779903412] 
len(x)

128

In [ ]:
samp_train = res_df1.sample(0.0003,seed = 1)

In [ ]:
samp_train

DataFrame[id: string, BERT_embed: vector, sarcasm: string, final_sent_class: string]

In [ ]:
samp_test

DataFrame[id: bigint, BERT_embed: vector, sarcasm: string, final_sent_class: string]

In [ ]:
samp_test = res_df.sample(0.00961,seed = 1)

In [ ]:
import pandas as pd
samp_train.toPandas().to_csv('/content/drive/MyDrive/Sentence Embeddings/samp_train_bert_tsne.csv',index = False,header = True)
samp_test.toPandas().to_csv('/content/drive/MyDrive/Sentence Embeddings/samp_test_bert_tsne.csv',index = False,header = True)

KeyboardInterrupt: ignored

In [ ]:
res_df.write.option("header",True).parquet("/content/drive/MyDrive/Sentence Embeddings/BERT SE 128/bert_embed_covid_dataset")

KeyboardInterrupt: ignored

In [ ]:
res_df1.write.option("header",True).parquet("/content/drive/MyDrive/Sentence Embeddings/BERT SE 128/bert_embed_dataset_final")

In [ ]:
df_res = spark.read.option("header",True).option("inferSchema",True).parquet("/content/drive/MyDrive/Sentence Embeddings/USE SE 512/use_embed_dataset_final")

1581420